<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 2.2.4 *The Google BigQuery UI and API*

# Introduction

The Google BigQuery UI provides access to Google's extensive collection of public data sets via an SQL-based query engine. 

The BigQuery API provides programmatic access to the data sets.

We can use the UI to discover interesting data before writing Python code to access it. Then we can reproduce it in an API request so as to aggregate large amounts of data on Google's infrastructure before pulling the results into our application.

## BigQuery Web UI

Work through the Quickstart at https://cloud.google.com/bigquery/docs/quickstarts/quickstart-web-ui.

You will need to set up a Google Cloud Platform account if you don't already have one. (This should not cost anything during the trial period unless you perform a large amount of querying. Afterwards, costs are based on actual resource usage, but most offerings have a free tier.)

## BigQuery API

You should already have the Google Cloud Client Library for Python installed (https://cloud.google.com/python/setup).

- Open Google Cloud Console (https://console.cloud.google.com/home/) and select to create a project.

- Under "Getting Started", select "Enable APIs and get credentials such as keys".

- In the API table, make sure the BigQuery API is enabled. Page back.

## Authentication

Go to https://cloud.google.com/docs/authentication/production and click the button to create a service account.

- Fill out the form, giving the account an appropriate name, and choose "Project Owner" for Account Type.

- Click "Create".

- The keys will get saved to your computer:

- Note the location and copy the file path (of the json file) to somewhere safe, for future reference.

- See here for more information:

https://cloud.google.com/iam/docs/understanding-service-accounts?&_ga=2.173177830.-495703703.1532572448#managing_service_account_keys 

This is supposed to get implicit key retrieval working:

- Windows:
    `set GOOGLE_APPLICATION_CREDENTIALS=[PATH]`
    
- Linux, MacOS:
    `export GOOGLE_APPLICATION_CREDENTIALS=[PATH]`
    
where `[PATH]` is the full file path of your json key file.

## Using the Python API

Google provides Python libraries for wrapping the Google APIs. For conda users, these are available on the "conda-forge" channel. (There are other Python libraries for Google APIs, possibly adding novel features or ease of use, but Google's is presumably the most current.)

__(Installing the "google-cloud-storage" and "google-cloud-bigquery" libraries should cover all the dependences for this lab.)__ 
If you have not installed these modules, run `conda install -c conda-forge google-cloud-storage google-cloud-bigquery` in a Terminal (Mac/Linux) or if on Windows, Anaconda Prompt (Anaconda), and follow the prompts.

In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import Dataset
from google.cloud import storage

If you have managed to get implicit key retrieval working, you can call `.Client()` with no argument:

In [2]:
storage_client = storage.Client()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

If you aren't so lucky, you need to invoke a method of the `.Client` object that takes the path to your key files as a string argument:

In [3]:
# put the path to your json key file here 
#,                    (or write code to load it from a file that your notebook can easily find)
key_path = r"C:\Users\Faizal Rahman\Downloads\tonal-works-297211-ce22506d867c.json"

This should not throw an error if key retrieval / assignemnt worked:

In [4]:
storage_client = storage.Client.from_service_account_json(key_path)

*Nb. The `storage` object was used in the above example, but there are other objects of interest that have polymorphic `Client` members that are used similarly, such as `bigquery`, which is used below.*

If implicit key retrieval is working for you, execute this:

In [ ]:
client = bigquery.Client()

if not, execute this:

In [5]:
client = bigquery.Client.from_service_account_json(key_path)

This client is associated with the default project (which was set or defaulted in the BigQuery UI): 

In [6]:
client.project

'tonal-works-297211'

A BigQuery project contains datasets. Datasets contain tables. To get at the data in a table we need to create a reference that covers this hierarchy; in the `bigquery` library this looks like `project.dataset.table`.  

(Nb. Queries can be performed on prjects and datasets, but most queries are performed on tables.)

To explore the public datasets we will start by reassgining our `client` variable using optional `project` parameter (set to `bigquery-public-data`):

In [7]:
#project = 'bigquery-public-data'
client = bigquery.Client.from_service_account_json(key_path, project = 'bigquery-public-data')
print(client.project)

bigquery-public-data


Here is how to get a list of the datasets in the current project:

In [8]:
datasets = list(client.list_datasets())
print(datasets)

[<google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F31216BEE0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F30EFBB2B0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D6100>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D6160>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D6490>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D64F0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D67F0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D6E80>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D6430>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122D6D90>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122FE3A0>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x000001F3122FE370>, <google.cloud.bigquery.dataset.DatasetListItem object at 0x0000

That wasn't helpful. We need to go deeper into the object structure to get at something meaningful. Actually, the `dataset_id` member contains the name attribute of a `dataset` object; write some code to print that name for each member of the list that was created above:

In [ ]:
#?

The google API objects in the `bigquery` library have their own overloads of the format() function that make them easier to read. Below is a function that exploits the `format` method of `project` and `dataset_id`, providing an easy way to list datasets:

In [9]:
# function for listing datasets in a project:
def printDatasetList(client):
    project = client.project    #: only one project can be associated with a client instance
    datasets = list(client.list_datasets())
    if datasets:
        print('Datasets in project {}:'.format(project))
        for dataset in datasets:  
            print('\t{}'.format(dataset.dataset_id))
        found = True
    else:
        print('{} project does not contain any datasets.'.format(project))
        found = False
    return found

In [12]:
# list datasets in the default project:
flag = printDatasetList(client)  #: assigning to `flag` suppresses printing the return value (normally `True`)

Datasets in project bigquery-public-data:
	austin_311
	austin_bikeshare
	austin_crime
	austin_incidents
	austin_waste
	baseball
	bitcoin_blockchain
	bls
	bls_qcew
	breathe
	broadstreet_adi
	catalonian_mobile_coverage
	catalonian_mobile_coverage_eu
	census_bureau_acs
	census_bureau_construction
	census_bureau_international
	census_bureau_usa
	census_utility
	cfpb_complaints
	chicago_crime
	chicago_taxi_trips
	cloud_storage_geo_index
	cms_codes
	cms_medicare
	cms_synthetic_patient_data_omop
	covid19_aha
	covid19_ecdc
	covid19_ecdc_eu
	covid19_geotab_mobility_impact
	covid19_geotab_mobility_impact_eu
	covid19_google_mobility
	covid19_google_mobility_eu
	covid19_govt_response
	covid19_italy
	covid19_italy_eu
	covid19_jhu_csse
	covid19_jhu_csse_eu
	covid19_nyt
	covid19_open_data
	covid19_open_data_eu
	covid19_public_forecasts
	covid19_public_forecasts_asia_ne1
	covid19_rxrx19
	covid19_symptom_search
	covid19_usafacts
	covid19_weathersource_com
	crypto_bitcoin
	crypto_bitcoin_cash
	crypto_da

This list should correspond to what is shown here https://bigquery.cloud.google.com/publicdatasets under the **bigquery-public-data** item:

Here is how to create a dataset reference object by assigning a project and a dataset name:

In [13]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id)

If our current project was something other than `bigquery-public-data`, we could still create this reference by specifying the project that contains the dataset:

In [14]:
dataset_id = 'samples'
dataset_ref = client.dataset(dataset_id, project = 'bigquery-public-data')

How can we get the path of the dataset?

In [ ]:
#?

Explore more of this object's members:

*(HINT: Jupyter Notebooks does not support code completion, but Spyder and other Python IDEs do. If you copy all the above code to a Python file within the IDE, you can type `dataset_ref.` in a new line, then hit the [Tab] key to see the available members for the object.)*

In [ ]:
#?
dataset_ref.

Here is a function for listing the tables in a dataset: 

In [15]:
# function for listing tables in a dataset:
def printTableList(client, dataset_id):
    project = client.project
    dataset_ref = client.dataset(dataset_id, project = project)    
    tables = list(client.list_tables(dataset_ref))
    if tables:
        print('Tables in dataset {}:'.format(dataset_id))
        for table in tables: 
            print('\t{}'.format(table.table_id))
        found = True
    else:
        print('{} dataset does not contain any tables.'.format(dataset_id))
        found = False
    return found

Use this function to list the tables in the current dataset:

In [16]:
#?
printTableList(client, dataset_id)

Tables in dataset samples:
	github_nested
	github_timeline
	gsod
	natality
	shakespeare
	trigrams
	wikipedia


True

To create a reference to a table within the dataset, we use the `table_id` attribute:

In [17]:
table_id = 'shakespeare'
table_ref = dataset_ref.table(table_id)

Check the name of the table that `table_ref` now points to:

In [18]:
#?
table_ref


TableReference(DatasetReference('bigquery-public-data', 'samples'), 'shakespeare')

To access the data in the table itself, we use the `get_table()` method:

In [19]:
table = client.get_table(table_ref)  # API Request

NOTE: The contents of the table are not actually in our memory after this call! We are working with a Big Data platform, now, and we could easily end up pulling GBs or TBs of data by accident. 

To minimise data bandwidth, memory consumption, and processing time, Big Data platforms employ ***lazy evaluation***. This means that no computation or data transfer actually takes place until we *realise* (use) the data. Even if we execute subsequent code that performs calculations on the data, no data flow or computation actually occurs until we request output (e.g. by executing a print to stdout or writing to a file). 

What kind of object is returned by `client.get_table`?

In [20]:
#?
type(table)


google.cloud.bigquery.table.Table

How can we view the design of the table (cOlumn names and types? The name of the boject attribute we need is the same term we learned in the module on databases:

In [21]:
#?
table.schema

[SchemaField('word', 'STRING', 'REQUIRED', 'A single unique word (where whitespace is the delimiter) extracted from a corpus.', (), None),
 SchemaField('word_count', 'INTEGER', 'REQUIRED', 'The number of times this word appears in this corpus.', (), None),
 SchemaField('corpus', 'STRING', 'REQUIRED', 'The work from which this word was extracted.', (), None),
 SchemaField('corpus_date', 'INTEGER', 'REQUIRED', 'The year in which this corpus was published.', (), None)]

Again, this is messy. If we wanted to refer to the column names and types in code, we might use something like this (which we could then parse into a dict):

In [22]:
result = ["{0} {1}".format(schema.name,schema.field_type) for schema in table.schema]
print(result)

['word STRING', 'word_count INTEGER', 'corpus STRING', 'corpus_date INTEGER']


But if we just want to print them, here is another neat function for that:

In [23]:
# function to print a table schema:
def printTableSchema(aTable):
    schemas = list(aTable.schema)
    if schemas:
        print('Table schema for {}:'.format(aTable.table_id))
        for aSchema in schemas:
            print('\t{0} {1}'.format(aSchema.name, aSchema.field_type))
        found = True
    else:
        found = False
    return found

Use this function to print the table schema:

In [24]:
#?
printTableSchema(table)


Table schema for shakespeare:
	word STRING
	word_count INTEGER
	corpus STRING
	corpus_date INTEGER


True

Now that we know what the columns are, we can write queries. Actually, we construct a query job by assigning an SQL statement to a method of the `client` object:

In [29]:
sql = """ SELECT COUNT(1) 
         FROM `bigquery-public-data.samples.shakespeare """
query_job = client.query(sql)

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/jobs?prettyPrint=false: Access Denied: Project bigquery-public-data: User does not have bigquery.jobs.create permission in project bigquery-public-data.

(job ID: 89c6766c-88fc-47d4-ad7f-a63cd9690649)

               -----Query Job SQL Follows-----               

    |    .    |    .    |    .    |    .    |    .    |
   1:SELECT COUNT(1) 
   2:         FROM `bigquery-public-data.samples.shakespeare 
    |    .    |    .    |    .    |    .    |    .    |

Why does this throw an error?

ANSWER: 

_____________________________________________________

So, what can we do?

In [32]:
#?

sql = """
         SELECT COUNT(1) 
         FROM `bigquery-public-data.samples.shakespeare;
         
         """
query_job = client.query(sql)
query_job.result

Forbidden: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/bigquery-public-data/jobs?prettyPrint=false: Access Denied: Project bigquery-public-data: User does not have bigquery.jobs.create permission in project bigquery-public-data.

(job ID: 0fc680a2-c31a-43f4-8382-6d32c13dd665)

               -----Query Job SQL Follows-----               

    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:         SELECT COUNT(1) 
   3:         FROM `bigquery-public-data.samples.shakespeare;
   4:         
   5:         
    |    .    |    .    |    .    |    .    |    .    |

If that worked, show what query_job is:

In [ ]:
#?


Once again, due to lazy execution, no actual execution occurs until we request output:

In [ ]:
for row in query_job:  # API request - fetches results
    print(row)

And, again, we need to manipulate this to make it neat. Each member of the rowset is a list and we only want to extract the value, which is in the first member:

In [ ]:
print(row[0])

So, we now know that this table has 164,656 rows. (We would not want to print it!)

A better coding practice is to write SQL statements that assign names (aliases) to derived values, so we don't forget what the resulting rowset contains. Rewite the above SQL statement so that the value returned is aliased a "num_rows", and assign the QueryJob as above:

In [ ]:
#?


Now we could use Python's `assert` statement to build a test into the first code block that operates on the rowset:

In [ ]:
for row in query_job:  # API request - fetches results
    # Row values can be accessed by field name or index:
    assert row[0] == row.num_rows == row['num_rows']  #: for debugging bad sql
    print(row.num_rows)

The above code checks that the name attribute of the value in `row[0]` is what we expected (i.e. "num_rows"). Also, it shows that we can refer to a field in a row by its object member `num_rows` or by using the same notation we use for Python dictionaries, `['num_rows']`. 

Write, execute, and print the results of a query that fetches 10 rows from the table, each containing the "word", "word_count", and "corpus" fields: 

In [ ]:
#?



(NOTE: Using `assert` religiously is good practice and will make debugging easier, but is probably overkill for non-production code.)

Whenever you catch yourself writing a swag of code to do something that seems rudimentary or low-level, there is a very good chance that you don't need to. A much easier way to handle the above requirement is to use the `to_dataframe` method of the QueryJob object:

In [ ]:
df = query_job.to_dataframe()
print(df)    

Although the above doesn't use `assert` (which you might still want to include in some test code), you will be able to tell at a glance if something is wrong with the contents of the DataFrame.

## Final Notes

1. Here is a readable way to code long SQL statements:

In [ ]:
sql = """
    SELECT word, word_count, corpus 
    FROM `bigquery-public-data.samples.shakespeare` 
    LIMIT 10
    """

2. If you had an application that needed to modify the tables or datasets in the `bigquery-public-data` is project, you could copy them to our own project, where you would have the permissions to do as you please with the data (subject to Google's terms of use).

3. We aren't limited to the datasets that are already in BigQuery. We can upload tables from our computer, and we can pull data in from other online souces. We will cover these tasks in another module.

## Next Steps

If you wish to pick up a few more skills you can go to https://cloud.google.com/bigquery/create-simple-app-api. (Note that we have already been through the preliminaries, so you can start at "Download the sample code".)

Alternatively, you can take a deeper dive into the API here: https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/usage.html.

© 2020 Institute of Data